In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
def get_df(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, header=None)
    df.columns = [
        "timestamp","index","uuid","utilization.gpu","memory.total","memory.used","memory.free","utilization.memory", "utilization.gpu2", "temperature.gpu","pstate"]
    df.loc[:, "timestamp"] = pd.to_datetime(df.timestamp).copy()
    t0 = df.loc[0, "timestamp"]
    df.loc[:, "timedelta"] = df.timestamp - t0
    df.loc[:, "timedelta_minutes"] = df.timedelta.dt.seconds / 60
    df.loc[:, "memory.used.gib"] = df.loc[:, "memory.used"] / 1024
    return df

In [ ]:
df = pd.concat((get_df("gpu-stats-wsinfer.csv").assign(pipeline="wsinfer"), get_df("gpu-stats-frontiers.csv").assign(pipeline="frontiers")))
df = df.set_index("timedelta")

In [ ]:
df_gpu_util_1min = df.groupby("pipeline")[["utilization.gpu", "memory.used", "memory.used.gib"]].resample('1Min').mean().reset_index()
df_gpu_util_1min.loc[:, "hours"] = df_gpu_util_1min.timedelta / np.timedelta64(1, "h")
df_gpu_util_1min.loc[:, "minutes"] = df_gpu_util_1min.timedelta / np.timedelta64(1, "m")

In [ ]:
sns.set_context("paper", font_scale=1.5)
# facet = sns.relplot(df_gpu_util_1min, x="hours", y="utilization.gpu", style="pipeline", style_order=["wsinfer", "frontiers"],kind="line",)
facet = sns.relplot(df_gpu_util_1min, x="hours", y="utilization.gpu", hue="pipeline", hue_order=["wsinfer", "frontiers"],kind="line",)
facet.ax.set_ylim(0, 100)
facet.ax.set_ylabel("GPU Utilization (%)")
facet.ax.set_xlabel("Relative Time (hours)")

sns.move_legend(facet, "upper center", bbox_to_anchor=(.7, .98), title=None)

plt.show()

In [ ]:
sns.set_context("paper", font_scale=1.5)
facet = sns.relplot(df_gpu_util_1min, x="hours", y="memory.used.gib", hue="pipeline", hue_order=["wsinfer", "frontiers"], kind="line")
# facet.ax.set_ylim(0, 100)
facet.ax.set_ylabel("GPU Memory Used (GiB)")
facet.ax.set_xlabel("Relative Time (hours)")

# facet.ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
# facet.ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
sns.move_legend(facet, "center right", bbox_to_anchor=(.68, .3), title=None)
plt.show()

In [ ]:
df.groupby("pipeline")["utilization.gpu"].describe()